### Getting optimal parameters for HNSW index

In this notebook, we will demonstrate how to find the optimal parameters for the HNSW index to achieve a high recall rate. Recall is a measure of the number of relevant items that are retrieved by a search algorithm. In the context of nearest neighbor search, recall is the proportion of true nearest neighbors that are returned in the top-k results. A recall of 1 indicates that all true nearest neighbors are retrieved, while a recall of 0 means that none of the true nearest neighbors are found.

We will use a synthetic dataset and perform a brute-force search to calculate the recall of the HNSW index with different parameter settings. The parameters we will tune are `M` (the number of neighbors to consider during construction), `ef` (the size of the dynamic list for the nearest neighbors), and `ef_construction` (the size of the dynamic list during construction). We will start with some initial values and iteratively adjust the parameters to increase the recall rate.

Let's start by importing the necessary libraries and generating the synthetic data.

In [ ]:
import hnswlib
import numpy as np

# Parameters
dim = 512  # Embedding dimension
num_elements = 27000  # Number of items (e.g., products)
num_queries = 100  # Number of queries to test
k = 10  # Number of nearest neighbors to retrieve

# Generate synthetic data
synthetic_data = np.random.random((num_elements, dim)).astype(np.float32)

# Function to calculate recall
def calculate_recall(M, ef, ef_construction):
    # Initialize HNSW index
    hnsw_index = hnswlib.Index(space='l2', dim=dim)
    hnsw_index.init_index(max_elements=num_elements, ef_construction=ef_construction, M=M)
    
    # Add synthetic data to the index
    hnsw_index.add_items(synthetic_data)
    
    # Initialize brute-force index for comparison
    bf_index = hnswlib.BFIndex(space='l2', dim=dim)
    bf_index.init_index(max_elements=num_elements)
    bf_index.add_items(synthetic_data)
    
    # Set search parameter for HNSW
    hnsw_index.set_ef(ef)
    hnsw_index.set_num_threads(4)
    
    # Generate random queries
    query_data = np.random.random((num_queries, dim)).astype(np.float32)
    
    # Perform knn search on both indices
    labels_hnsw, _ = hnsw_index.knn_query(query_data, k)
    labels_bf, _ = bf_index.knn_query(query_data, k)
    
    # Calculate recall
    correct = sum([1 for i in range(num_queries) for l in labels_hnsw[i] if l in labels_bf[i]])
    recall = float(correct) / (k * num_queries)
    
    return recall

# Initial parameter values
M = 16
ef = 100
ef_construction = 200

# Measure initial recall
initial_recall = calculate_recall(M, ef, ef_construction)
print(f"Initial recall with M={M}, ef={ef}, ef_construction={ef_construction}: {initial_recall:.4f}")

# Iteratively adjust parameters to increase recall
target_recall = 0.95  # Target recall close to 1
step_size = 50
max_ef = 1000
while initial_recall < target_recall and ef <= max_ef:
    # Increase ef
    ef += step_size
    initial_recall = calculate_recall(M, ef, ef_construction)
    print(f"Adjusted recall with M={M}, ef={ef}, ef_construction={ef_construction}: {initial_recall:.4f}")

# If recall is still not high enough, adjust M
if initial_recall < target_recall:
    print("\nIncreasing M for further optimization...")
    M = 32
    ef = 200
    ef_construction = 300
    
    initial_recall = calculate_recall(M, ef, ef_construction)
    print(f"Recall after increasing M to {M} with ef={ef}, ef_construction={ef_construction}: {initial_recall:.4f}")

    # Further adjust ef
    while initial_recall < target_recall and ef <= max_ef:
        ef += step_size
        initial_recall = calculate_recall(M, ef, ef_construction)
        print(f"Adjusted recall with M={M}, ef={ef}, ef_construction={ef_construction}: {initial_recall:.4f}")
    
print("\nFinal parameter settings to achieve recall close to 1:")
print(f"M = {M}, ef = {ef}, ef_construction = {ef_construction}, Recall = {initial_recall:.4f}")


Initial recall with M=16, ef=100, ef_construction=200: 0.4400
